In [1]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             52G     0   52G   0% /dev
tmpfs            11G  608K   11G   1% /run
/dev/sda1       148G   44G   98G  32% /
tmpfs            52G     0   52G   0% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
/dev/sda15      124M   11M  114M   9% /boot/efi
/dev/sdb        246G  6.5G  239G   3% /home/jupyter


In [2]:
!pip install --upgrade pip
!pip install kaggle
!pip install kagglehub
!pip install --upgrade kaggle

In [4]:
#you need to upload your kaggle.json file to the current directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory


In [9]:
import concurrent.futures
import os
import requests
import zipfile
from tqdm import tqdm

print(os.getcwd())

D:\DeepFakeSpeech\Notebooks


In [8]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

In [10]:
api.dataset_download_files('mathurinache/the-lj-speech-dataset', path='../Data', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/mathurinache/the-lj-speech-dataset


In [11]:
api.dataset_download_files('abdallamohamed312/in-the-wild-dataset', path='../Data', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/abdallamohamed312/in-the-wild-dataset


In [18]:
api.dataset_download_files('namhocayai/voxceleb1', path='../Data', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/namhocayai/voxceleb1


In [19]:
api.dataset_download_files('walimuhammadahmad/fakeaudio', path='../Data', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/walimuhammadahmad/fakeaudio


In [ ]:
zip_files = [
    ("https://nextcloud.fit.vutbr.cz/s/HYcHPMTNn753QLF/download/dataset_01.zip", "dataset_01"),
    ("https://nextcloud.fit.vutbr.cz/s/cHAZzzkwCnSGBEX/download/dataset_02.zip", "dataset_02"),
    ("https://nextcloud.fit.vutbr.cz/s/EqHWL5G7BnfgKKC/download/dataset_03.zip", "dataset_03"),
    ("https://nextcloud.fit.vutbr.cz/s/DZdPWXySKbXEQSg/download/dataset_04.zip", "dataset_04"),
    ("https://nextcloud.fit.vutbr.cz/s/zKYSwFAHeXGG7YF/download/metadata.zip", "metadata")
]

base_dest_path = "../Data/diffusion_dataset/"
os.makedirs(base_dest_path, exist_ok=True)


def download_file(url, output_path):
    response = requests.get(url, stream=True)
    if response.status_code != 200:
        raise Exception(f"下载失败: {url}, 状态码: {response.status_code}")

    total_size = int(response.headers.get("content-length", 0))
    chunk_size = 1024
    with open(output_path, "wb") as f, tqdm(
            total=total_size, unit="B", unit_scale=True,
            desc=os.path.basename(output_path)
    ) as pbar:
        for chunk in response.iter_content(chunk_size=chunk_size):
            if chunk:
                f.write(chunk)
                pbar.update(len(chunk))
    return output_path


def extract_zip(zip_path, extract_to):
    try:
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"{os.path.basename(zip_path)} 解压完成！")
    except zipfile.BadZipFile:
        print(f"错误：{os.path.basename(zip_path)} 不是有效的 zip 文件。")


def download_and_extract(item):
    url, folder_name = item
    dest_path = os.path.join(base_dest_path, folder_name)
    os.makedirs(dest_path, exist_ok=True)

    zip_path = os.path.join(dest_path, url.split('/')[-1])

    if not os.path.exists(zip_path):
        try:
            download_file(url, zip_path)
        except Exception as e:
            print(f"{folder_name} 下载失败: {e}")
            return folder_name, False
    else:
        print(f"{folder_name} 的文件已存在，跳过下载。")

    extract_zip(zip_path, dest_path)
    return folder_name, True


with concurrent.futures.ThreadPoolExecutor(max_workers=32) as executor:
    futures = [executor.submit(download_and_extract, item) for item in zip_files]
    for future in concurrent.futures.as_completed(futures):
        folder, success = future.result()
        if success:
            print(f"{folder} 下载并解压成功！")
        else:
            print(f"{folder} 处理失败。")




t_03.zip:   0%|          | 0.00/16.6G [00:00<?, ?B/s]
dataset_01.zip:   0%|          | 0.00/9.51G [00:00<?, ?B/s]

dataset_01.zip:   0%|          | 50.2k/9.51G [00:00<7:33:03, 350kB/s]]
aset_04.zip:   0%|          | 50.2k/13.1G [00:00<10:44:43, 338kB/s]


2.zip:   0%|          | 0.00/9.34G [00:00<?, ?B/s]



:   0%|          | 0.00/24.4M [00:00<?, ?B/s]

dataset_01.zip:   0%|          | 86.0k/9.51G [00:00<9:33:39, 276kB/s]]
aset_04.zip:   0%|          | 85.0k/13.1G [00:00<13:36:12, 267kB/s]


2.zip:   0%|          | 50.2k/9.34G [00:00<7:54:39, 328kB/s]



dataset_01.zip:   0%|          | 157k/9.51G [00:00<7:22:09, 358kB/s] 

t_03.zip:   0%|          | 157k/16.6G [00:00<12:57:32, 356kB/s] 
aset_04.zip:   0%|          | 140k/13.1G [00:00<11:48:15, 308kB/s] 


2.zip:   0%|          | 84.0k/9.34G [00:00<9:55:33, 261kB/s]



dataset_01.zip:   0%|          | 312k/9.51G [00:00<4:18:34, 613kB/s]

t_03.zip:   0%|          | 312k/16.6G [00:00<7:33:51, 610kB/s] 
aset_04.zip:   0%|          | 25

metadata.zip 解压完成！
metadata 下载并解压成功！


dataset_01.zip:   1%|          | 49.2M/9.51G [00:04<12:47, 12.3MB/s]

t_03.zip:   0%|          | 51.6M/16.6G [00:04<21:00, 13.1MB/s]
aset_04.zip:   0%|          | 47.6M/13.1G [00:04<16:22, 13.3MB/s]


dataset_01.zip:   1%|          | 51.3M/9.51G [00:04<12:26, 12.7MB/s]
aset_04.zip:   0%|          | 49.7M/13.1G [00:04<16:20, 13.3MB/s]

t_03.zip:   0%|          | 53.7M/16.6G [00:05<21:03, 13.1MB/s]


dataset_01.zip:   1%|          | 53.4M/9.51G [00:05<12:14, 12.9MB/s]

t_03.zip:   0%|          | 55.8M/16.6G [00:05<20:44, 13.3MB/s]
aset_04.zip:   0%|          | 51.8M/13.1G [00:05<16:18, 13.3MB/s]


dataset_01.zip:   1%|          | 55.5M/9.51G [00:05<12:03, 13.1MB/s]

t_03.zip:   0%|          | 57.9M/16.6G [00:05<20:43, 13.3MB/s]
aset_04.zip:   0%|          | 53.9M/13.1G [00:05<16:16, 13.3MB/s]


dataset_01.zip:   1%|          | 57.6M/9.51G [00:05<11:58, 13.2MB/s]
aset_04.zip:   0%|          | 55.9M/13.1G [00:05<16:15, 13.4MB/s]

t_03.zip:   0%|          | 60.0M/16.6G [00:05<20:50, 13.2MB/

In [6]:
import os

base_dir = "./Data/diffusion_dataset/"

# 用于存储每个文件夹下的 .wav 文件数量
folder_wav_count = {}

# 递归遍历目录树
for root, dirs, files in os.walk(base_dir):
    # 计算当前文件夹中 .wav 文件数量（不区分大小写）
    wav_count = sum(1 for file in files if file.lower().endswith('.wav'))
    # 获取相对于 base_dir 的路径
    rel_path = os.path.relpath(root, base_dir)
    # 如果是顶层目录，则以文件夹名称显示
    if rel_path == ".":
        rel_path = os.path.basename(os.path.abspath(base_dir))
    folder_wav_count[rel_path] = wav_count

# 打印结果：每个文件夹名称和对应的 .wav 文件数量
for folder, count in folder_wav_count.items():
    print(f"{folder}: {count}")


diffusion_dataset: 0
dataset_04: 0
dataset_04/fast_pitch: 13100
dataset_04/vits: 13100
dataset_04/tacotron2-DCA: 13100
dataset_04/glow-tts: 13100
dataset_01: 0
dataset_01/DiffGAN-TTS_aux: 13100
dataset_01/DiffGAN-TTS_shallow: 13100
dataset_01/DiffGAN-TTS_naive: 13100
dataset_03: 0
dataset_03/wavegrad2: 13100
dataset_03/tacotron2-DCA_bddm: 13100
dataset_03/tacotron2-DCA_diffwave: 13100
dataset_03/tacotron2-DCA_wavegrad: 13100
dataset_02: 0
dataset_02/NATSpeech_DiffSpeech: 13100
dataset_02/ProDiff: 13100
dataset_02/Grad-TTS: 13100
metadata: 0
metadata/protocols: 0
metadata/protocols/.ipynb_checkpoints: 0
metadata/detection scores: 0
metadata/detection scores/IDSD: 0
metadata/detection scores/SSL_antispoofing_LA: 0
metadata/detection scores/LCNN: 0
metadata/quality: 0
metadata/quality/.ipynb_checkpoints: 0
metadata/rtf: 0


In [8]:
import os


def print_directory_tree(start_path, indent=""):
    # 打印当前目录名称
    print(indent + os.path.basename(start_path))
    # 获取目录下所有项目，并排序
    for item in sorted(os.listdir(start_path)):
        path = os.path.join(start_path, item)
        if os.path.isdir(path):
            # 如果是文件夹则递归打印
            print_directory_tree(path, indent + "    ")


# 指定包含 dataset1-4 的基础目录
base_path = "./Data/diffusion_dataset"

# 列出需要展示的文件夹名称
datasets = ["dataset_01", "dataset_02", "dataset_03", "dataset_04"]

for d in datasets:
    folder_path = os.path.join(base_path, d)
    if os.path.exists(folder_path):
        print_directory_tree(folder_path)
        print()  # 空行分隔不同数据集的结构
    else:
        print(f"{folder_path} 不存在。")


dataset_01
    DiffGAN-TTS_aux
    DiffGAN-TTS_naive
    DiffGAN-TTS_shallow

dataset_02
    Grad-TTS
    NATSpeech_DiffSpeech
    ProDiff

dataset_03
    tacotron2-DCA_bddm
    tacotron2-DCA_diffwave
    tacotron2-DCA_wavegrad
    wavegrad2

dataset_04
    fast_pitch
    glow-tts
    tacotron2-DCA
    vits

